In [1]:
import os
import sys
import json
import pandas as pd

In [2]:
import copy
import os
import re
from typing import Union, Tuple

import numpy as np

In [3]:
def add_path(path):
    if path not in sys.path:
        sys.path.append(path)

In [4]:
root = os.getcwd().split("NatGen")[0]
root

'/home/mdrahman/projects/'

In [5]:
add_path(f"{root}NatGen")

In [6]:
from src.data_preprocessors import *

from src.data_preprocessors.language_processors import JavaAndCPPProcessor
from src.data_preprocessors.transformations.transformation_base import TransformationBase

language = 'c'
parser_path = root + '/NatGen/parser/languages.so'


In [443]:
import re
import copy

import numpy as np
from tree_sitter import Node

from src.data_preprocessors.language_processors.utils import get_tokens, dfs_print


class JavaAndCPPProcessor:

    @classmethod
    def extract_if_else(cls, root, code_str, operator_list):
        ext_opt_list = ["&&", "&", "||", "|"]
        expressions = []
        queue = [root]

        not_consider = []
        while len(queue) > 0:
            current_node = queue[0]
            queue = queue[1:]
            if str(current_node.type) == 'if_statement':
                clause = code_str[current_node.start_byte:current_node.end_byte].decode()
                des = (current_node.children)[1]
                cond = code_str[des.start_byte:des.end_byte].decode()
                stack = [des]
                nodes = []
                while len(stack) > 0:
                    root1 = stack.pop()
                    if len(root1.children) == 0:
                        nodes.append(root1)

                    for child in root1.children:
                        stack.append(child)
                nodes.reverse()
                counter = 0
                extra_counter = 0
                for w in nodes:
                    if str(w.type) in operator_list:
                        counter = counter + 1
                    if str(w.type) in ext_opt_list:
                        extra_counter = extra_counter + 1
                if not (counter == 1 and extra_counter == 0):
                    continue
                children_nodes = current_node.children
                flagx = 0
                flagy = 0
                for w in children_nodes:
                    if str(w.type) == "else":
                        flagx = 1
                    if str(w.type) == "if_statement":
                        not_consider.append(w)
                        flagy = 1
                if flagx == 1 and flagy == 0:
                    expressions.append([current_node, des])
            for child in current_node.children:
                if child not in not_consider:
                    queue.append(child)

        return expressions
    
    @classmethod
    def get_tokens_for_multi_blockswap(cls, code, root, block_change, parent=None, flag=0, mp=None):
        if isinstance(code, str):
            code = code.encode()
        assert isinstance(root, Node)
        tokens = []
        
        if root.type == "comment":
            return tokens, None
        if "string" in str(root.type):
            return [code[root.start_byte:root.end_byte].decode()], None
        children = root.children
        if len(children) == 0:
            if root in block_change[2]:
                op = code[root.start_byte:root.end_byte].decode()
                if op == "<":
                    tokens.append(">=")
                elif op == ">":
                    tokens.append('<=')
                elif op == ">=":
                    tokens.append('<')
                elif op == "<=":
                    tokens.append('>')
                elif op == "!=":
                    tokens.append('==')
                elif op == "==":
                    tokens.append('!=')
            else:
                if root.type == 'identifier':
                    if not flag:
                        text = code[root.start_byte:root.end_byte].decode()
                        le = (len(text)) // 2
                        # print(type(le))
                        mp[text] = text[:le] + '_' + text[le:]
                tokens.append(code[root.start_byte:root.end_byte].decode())
        # if parent is not None and parent.type == 'switch_statement' and root.type=='compound_statement':
        #     case_positions = []
        #     for i, child in enumerate(children):
        #         if child.type == 'case_statement' and ('break' in code[child.start_byte:child.end_byte].decode()):
        #             case_positions.append(i)
        #     case_positions = case_positions[:-1]
        #     copied = copy.deepcopy(case_positions)
        #     np.random.shuffle(copied)
        #     # print(code[case_positions[-1].start_byte: case_positions[-1].end_byte])
        #     # print(children)
        #     # print(root.type, code[root.start_byte:root.end_byte].decode())
        #     for i, child in enumerate(children):
        #         child_type = str(child.type)
        #         # print(child, child_type)
        #         if child_type == 'case_statement' and (i in copied):
        #             idx = copied.index(i)
        #             ts, _ = cls.get_tokens_for_multi_blockswap(code, children[case_positions[idx]], block_change, parent=root)
        #         else:
        #             ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root)
        #         tokens += ts
        nflag = flag | (1 if root.type == 'call_expression' else 0)
        if root.type == 'binary_expression' and len(children) == 3 and children[1].type in ('+', '*'):
            for child in children[::-1]:
                ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root, flag=nflag, mp=mp)
                tokens += ts
        elif root.type == 'if_statement' and len(children) in (3, 5):
            # assert len(children) in (3, 5), code[root.start_byte:root.end_byte].decode()
            for id_ in range(2):
                ts, _ = cls.get_tokens_for_multi_blockswap(code, children[id_], block_change, parent=root, flag=nflag, mp=mp)
                tokens += ts
            if children[2].type != 'compound_statement':
                tokens.append('{')
            ts, _ = cls.get_tokens_for_multi_blockswap(code, children[2], block_change, parent=root, flag=nflag, mp=mp)
            tokens += ts
            if children[2].type != 'compound_statement':
                tokens.append('}')
            if len(children) == 5:
                ts, _ = cls.get_tokens_for_multi_blockswap(code, children[3], block_change, parent=root, flag=nflag, mp=mp)
                tokens += ts
                if children[4].type != 'compound_statement':
                    tokens.append('{')
                ts, _ = cls.get_tokens_for_multi_blockswap(code, children[4], block_change, parent=root, flag=nflag, mp=mp)
                tokens += ts
                if children[4].type != 'compound_statement':
                    tokens.append('}')
        else:
            for child in children:
                child_type = str(child.type)
                ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root, flag=nflag, mp=mp)

                # # print(child, child_type)
                # if child in block_change[0]:
                #     idx = block_change[0].index(child)
                #     ts, _ = cls.get_tokens_for_multi_blockswap(code, block_change[1][idx], block_change, parent=root)
                # else:
                #     ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root)
                tokens += ts
        # if root.type == 'declaration':
        #     text = code[root.start_byte:root.end_byte].decode()
        #     if '=' not in text:
        # print(parent.type)
        # if parent is not None and parent.type == 'switch_statement' and root.type=='compound_statement':
        #     print(children)
        #     print(root.type, code[root.start_byte:root.end_byte].decode())
        return tokens, None


    @classmethod
    def multi_block_swap_c(cls, code_str, parser):
        code = code_str.encode()
        root = parser.parse_code(code)
        operator_list = ['<', '>', '<=', '>=', '==', '!=']
        pair = cls.extract_if_else(root, code, operator_list)
        success = False
        lst = list(range(0, len(pair)))
        try:
            block_change = [[], [], []]
            for selected in lst:
                clause = pair[selected][0]
                des = pair[selected][1]
                st = [des]
                nodes = []
                while len(st) > 0:
                    root1 = st.pop()
                    if len(root1.children) == 0:
                        nodes.append(root1)
                        if (code[root1.start_byte:root1.end_byte].decode()) in operator_list:
                            opt_node = root1
                            break
                    for child in root1.children:
                        st.append(child)
                nodes = clause.children
                flag = 0
                first_block, second_block = None, None
                for current_node in nodes:
                    if str(current_node.type) == 'compound_statement':
                        if flag == 0:
                            first_block = current_node
                            flag = 1
                        else:
                            second_block = current_node
                if first_block and second_block:
                    block_change[0].append(first_block)
                    block_change[1].append(second_block)
                    block_change[0].append(second_block)
                    block_change[1].append(first_block)
                    block_change[2].append(opt_node)
            try:
                mp = {}
                code_list = \
                    cls.get_tokens_for_multi_blockswap(code, root, block_change, None, 0, mp)[0]
                for el in mp:
                    while (el in code_list):
                        idx = code_list.index(el)
                        code_list[idx] = mp[el]
                code_string = ""
                for w in code_list:
                    code_string = code_string + w + " "
                code_string = code_string.strip()
                success = True
            except Exception as E:
                print("Exception: ", str(E))
                success = False
        except Exception as E:
            print("Exception occurred", str(E))
            pass
        if not success:
            code_string = cls.beautify_java_code(get_tokens(code_str, root))
        return code_string, success


In [444]:


processor_function = {"c": [JavaAndCPPProcessor.multi_block_swap_c],}


class BlockSwap(TransformationBase):
    """
    Swapping if_else block
    """

    def __init__(self, parser_path, language):
        super(BlockSwap, self).__init__(parser_path=parser_path, language=language)
        self.language = language
        self.transformations = processor_function[language]
        processor_map = {
            "c": self.get_tokens_with_node_type,
        }
        self.final_processor = processor_map[self.language]

    def transform_code(
            self,
            code: Union[str, bytes],
    ) -> Tuple[str, object]:
        success = False
        transform_functions = copy.deepcopy(self.transformations)
        while not success and len(transform_functions) > 0:
            function = np.random.choice(transform_functions)
            transform_functions.remove(function)
            modified_code, success = function(code, self)
            if success:
                code = modified_code
        root_node = self.parse_code(
            code=code
        )
        return_values = self.final_processor(
            code=code.encode(),
            root=root_node
        )
        if isinstance(return_values, tuple):
            tokens, types = return_values
        else:
            tokens, types = return_values, None
        # print(tokens)
        return  " ".join(tokens), \
               {
                   "types": types,
                   "success": success
               }


In [445]:
c_code = """
#include<stdio.h>
#define y  9
int process(int option, int a, int b) {
    int *x;
    int k = a + b;
    int result = k;

    if (b != 0) {
        result = a / b;
    } else if(a < 0) result = -a /b;
    else if (b == 5) result = 2;
    else if (k == 2) result = 0;
    else {
        printf("Error: Division by zero is undefined.\n");
        return -1;  // Return an error code
    }
    switch (option) {
        case 1:
            result = a + b * c;
            break;
        case 2:
            result = ((a + b) - c);
            break;
        case 3:
            result = a * c;
            break;
        default:

            if (b != 0) {
                result = a / b;
            } else if(a < 0) result = -a /b;
            else if (b == 5) result = 2;
            else if (k == 2) result = 0;
            else {
                printf("Error: Division by zero is undefined.\n");
                return -1;  // Return an error code
            }
            printf("Error: Invalid option. Please choose a valid option (1-4).\n");
            return -1;  // Return an error code
    }
    
    return result;
}
"""

In [446]:
' '.join(c_code.split())

'#include<stdio.h> #define y 9 int process(int option, int a, int b) { int *x; int k = a + b; int result = k; if (b != 0) { result = a / b; } else if(a < 0) result = -a /b; else if (b == 5) result = 2; else if (k == 2) result = 0; else { printf("Error: Division by zero is undefined. "); return -1; // Return an error code } switch (option) { case 1: result = a + b * c; break; case 2: result = ((a + b) - c); break; case 3: result = a * c; break; default: if (b != 0) { result = a / b; } else if(a < 0) result = -a /b; else if (b == 5) result = 2; else if (k == 2) result = 0; else { printf("Error: Division by zero is undefined. "); return -1; // Return an error code } printf("Error: Invalid option. Please choose a valid option (1-4). "); return -1; // Return an error code } return result; }'

In [447]:
def beautify(code):
    space = ''
    final_code = ''
    flag = False
    string_on = False
    case_on = False
    for ch in code:
        # print(ch)
        if string_on:
            final_code += ch
            if ch == "\"":
                string_on = False
            continue
        elif ch == "\"":
            final_code += "\""
            string_on = True
            continue
        if ch == '{':
            space += '\t'
            final_code += "{\n"
            flag = True
        elif ch == '}':
            space = space[:-1]
            final_code += "\n" + space + ch + "\n"
            flag = True
        elif ch == ';':
            if len(final_code) > 0 and final_code[-1] == '\n':
                final_code = final_code[:-1]
            final_code += ch + ";\n"
            flag = True
        elif ch == ':':
            final_code += ch + "\n"
            flag = True
        else:
            if flag:
                final_code += space
                flag = False
            final_code += ch
    return final_code



In [448]:
transformer = BlockSwap(parser_path, language)

In [449]:
code, _ = transformer.transform_code(c_code)

In [450]:
final_code = beautify(code)

In [451]:
print(final_code)

#include <stdio.h> 
 #define _y    9  
 int pro_cess ( int opt_ion , int _a , int _b ) {
	 int * _x ;;
	 int _k = _b + _a ;;
	 int res_ult = _k ;;
	 if ( _b != 0 ) {
		 res_ult = _a / _b ;;
		 
	}
	 else {
		 if ( _a < 0 ) {
			 res_ult = - _a / _b ;;
			 
		}
		 else {
			 if ( _b == 5 ) {
				 res_ult = 2 ;;
				 
			}
			 else {
				 if ( _k == 2 ) {
					 res_ult = 0 ;;
					 
				}
				 else {
					 printf ( "Error: Division by zero is undefined.
" ) ;;
					 return -1 ;;
					 
				}
				 
			}
			 
		}
		 
	}
	 switch ( opt_ion ) {
		 case 1 :
		 res_ult = _c * _b + _a ;;
		 break ;;
		 case 2 :
		 res_ult = ( ( _b + _a ) - _c ) ;;
		 break ;;
		 case 3 :
		 res_ult = _c * _a ;;
		 break ;;
		 default :
		 if ( _b != 0 ) {
			 res_ult = _a / _b ;;
			 
		}
		 else {
			 if ( _a < 0 ) {
				 res_ult = - _a / _b ;;
				 
			}
			 else {
				 if ( _b == 5 ) {
					 res_ult = 2 ;;
					 
				}
				 else {
					 if ( _k == 2 ) {
						 res_ult = 0 ;;
						 
					}
					 else {
					

In [452]:
test_data_path = '../../data/Devign/test.jsonl'
f = open(test_data_path, 'r')
data = [json.loads(ex) for ex in f]
f.close()

In [453]:
for i in range(len(data)):
    # print(data[i]['func'])
    data[i]['func'] = beautify(transformer.transform_code(data[i]['func'])[0])
    # if i == 5:
    #     break

In [454]:
test_data_path = '../../data/Devign/test_space_sc_bracket_uc.jsonl'
f = open(test_data_path, 'w')
f.write('\n'.join([json.dumps(ex) for ex in data]))
f.close()

In [267]:
from tree_sitter import Language, Parser

In [393]:
lang_object = Language(parser_path, language)
parser = Parser()
parser.set_language(lang_object)

In [422]:
def walk(root, code, flag, mp):
    if isinstance(code, str):
        code = code.encode()
    if "string" in str(root.type):
        return [code[root.start_byte:root.end_byte].decode()]
    children = root.children

    if len(children) == 0:
        text = code[root.start_byte:root.end_byte].decode()
        if root.type == 'identifier':
            if not flag:
                le = (len(text)) // 2
                # print(type(le))
                mp[text] = text[:le] + '_' + text[le:]
        return [text]
    # if "declarataion" in root.type:
    #     print(root.type, code[root.start_byte:root.end_byte].decode())
    # print(children)
    print(root.type, code[root.start_byte:root.end_byte].decode())
    for child in children:
        # if "declaration" in root.type:
        print(child.type, code[child.start_byte:child.end_byte].decode())
        # print(child)
        # if "binary_expression" in root.type and len(children) == 3:
        # #     print("child", code[child.start_byte: child.end_byte].decode())
        # else:
    ts = []
    nflag = flag | (1 if root.type == 'call_expression' else 0)
    for child in children:
        # if "declaration" in root.type:
        # print(child.type, code[child.start_byte:child.end_byte].decode())
        # print(child)
        # if "binary_expression" in root.type and len(children) == 3:
        # #     print("child", code[child.start_byte: child.end_byte].decode())
        # else:

        ts += walk(child, code, nflag, mp)
    return ts

In [431]:
c_code = """

int process(int option, int a, int b) {

    if (b != 0) {
        result = a / b;
    } else if(a < 0) result = -a /b;
    else {
        printf("Error: Division by zero is undefined.\n");
        return -1;  // Return an error code
    }
    int x, y;
    memset(x, y);
    int k = add(4, 5);

}
"""

In [432]:
tree =parser.parse(c_code.encode())

In [433]:
mp = {}
tokens = walk(tree.root_node, c_code, 0, mp)
for el in mp:
    while (el in tokens):
        idx = tokens.index(el)
        tokens[idx] = mp[el]

translation_unit int process(int option, int a, int b) {

    if (b != 0) {
        result = a / b;
    } else if(a < 0) result = -a /b;
    else {
        printf("Error: Division by zero is undefined.
");
        return -1;  // Return an error code
    }
    int x, y;
    memset(x, y);
    int k = add(4, 5);

}

function_definition int process(int option, int a, int b) {

    if (b != 0) {
        result = a / b;
    } else if(a < 0) result = -a /b;
    else {
        printf("Error: Division by zero is undefined.
");
        return -1;  // Return an error code
    }
    int x, y;
    memset(x, y);
    int k = add(4, 5);

}
function_definition int process(int option, int a, int b) {

    if (b != 0) {
        result = a / b;
    } else if(a < 0) result = -a /b;
    else {
        printf("Error: Division by zero is undefined.
");
        return -1;  // Return an error code
    }
    int x, y;
    memset(x, y);
    int k = add(4, 5);

}
primitive_type int
function_declarator process(int 

In [434]:
print(beautify(' '.join(tokens)))

int pro_cess ( int opt_ion , int _a , int _b ) {
	 if ( _b != 0 ) {
		 res_ult = _a / _b ;;
		 
	}
	 else if ( _a < 0 ) res_ult = - _a / _b ;;
	 else {
		 printf ( "Error: Division by zero is undefined.
" ) ;;
		 return -1 ;;
		 // Return an error code 
	}
	 int _x , _y ;;
	 memset ( _x , _y ) ;;
	 int _k = add ( 4 , 5 ) ;;
	 
}

